In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [6]:
# ENV SET UP
# python > 3.9 recommended
# git clone git@github.com:facebookresearch/llama-recipes.git
# cd llama-recipes
# pip install -r requirements.txt
# pip install psy_llm_chat
# pip install openai

In [7]:
# MODEL hf creation
# The first three model_names are local huggingface versions of the llama models, i.e. after downloading the meta weights, they were created following 
# the llama-recipes quickstart.ipynb directions, from: https://github.com/facebookresearch/llama-recipes/blob/main/quickstart.ipynb

# i.e.:
# %%bash
# pip install transformers datasets accelerate sentencepiece protobuf==3.20 py7zr scipy peft bitsandbytes fire torch_tb_profiler ipywidgets
# TRANSFORM=`python -c "import transformers;print('/'.join(transformers.__file__.split('/')[:-1])+'/models/llama/convert_llama_weights_to_hf.py')"`
# print(TRANSFORM)
# python ${TRANSFORM} --input_dir ./models/llama-2-70b-chat --model_size 70B --output_dir ./models_hf/70B_chat

# NB, for the above code to work, meta weights need to be stored in a `models` dir using the following structure, where
# each model is stored in a sub-directory in <<input_dir>> which matches the --model_size param, e.g. 7B, 13B, 70B:
# ├── models
# │   ├── llama-2-13b
# │   │   └── 13B
# │   ├── llama-2-13b-chat
# │   │   └── 13B
# │   ├── llama-2-70b
# │   │   └── 70B
# │   ├── llama-2-70b-chat
# │   │   └── 70B
# │   ├── llama-2-7b
# │   │   └── 7B
# │   └── llama-2-7b-chat
# │       └── 7B

In [13]:
# import from psy_llm_chat class to chat with GPT and Llama
import psy_llm_chat as plc
import openai

In [4]:
# Define a simple dialog
test_dialog = [
    # {'role': 'system', 'content': 'Always answer with Haiku'},
    {'role': 'user', 'content': "Can you use TiKZ code to draw a horse?"}
]

In [5]:
#  #local models 
model_name ="./llama_models/models_hf/7B_chat"
# model_name ="./llama_models/models_hf/13B_chat"
# model_name ="./llama_models/models_hf/70B_chat"

# # huggingface
# models can also be loaded from HF hub (once access has been granted), e.g.:
# model_name = "meta-llama/Llama-2-7b-chat-hf"
# model_name = "meta-llama/Llama-2-13b-chat-hf"
# model_name = "meta-llama/Llama-2-70b-chat-hf"

# load the model defined by <<model_name>>, must set output_hidden_states and ...attentions to True to access these later.
lc = plc.LlamaChat(model_name, output_hidden_states=True, output_attentions=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
# run the chat by calling the class
# to end the chat, input `END_THIS_NOW`; this terminates the chat and returns the dialog 
result = lc(test_dialog)

User: Can you use TiKZ code to draw a horse?


/home/coler/anaconda3/envs/llama/lib/python3.10/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


Assistant: Yes, you can use TiKZ code to draw a horse. Here is an example of a simple horse drawing using TiKZ:

\documentclass{article}
\usepackage{tikz}
\usetikzlibrary{shapes,arrows,positioning}
\begin{document}
\beginIGROUP\draw (0,0) rectangle (2,2);% body
\draw (1,1) circle (0.5ex);% head
\draw (1,1.5) -- (2,0.5);% neck
\draw (0.5,1) -- (1.5,0);% tail
\endtikzpicture
\end{document}
This will produce a simple horse drawing with a body, head, neck, and tail. You can customize the appearance of the horse by adjusting the dimensions of the shape, adding more details such as eyes, mane, and hooves, or using different shapes and styles.

You can also use the `pic` command to include a predefined horse shape in your drawing:
\documentclass{article}
\usepackage{tikz}
\usetikzlibrary{shapes.animals}
\


User:  END_THIS_NOW


In [7]:
# dialog is a list of dicts
result

[{'role': 'user', 'content': 'Can you use TiKZ code to draw a horse?'},
 {'role': 'assistant',
  'content': 'Yes, you can use TiKZ code to draw a horse. Here is an example of a simple horse drawing using TiKZ:\n\n\\documentclass{article}\n\\usepackage{tikz}\n\\usetikzlibrary{shapes,arrows,positioning}\n\\begin{document}\n\\beginIGROUP\\draw (0,0) rectangle (2,2);% body\n\\draw (1,1) circle (0.5ex);% head\n\\draw (1,1.5) -- (2,0.5);% neck\n\\draw (0.5,1) -- (1.5,0);% tail\n\\endtikzpicture\n\\end{document}\nThis will produce a simple horse drawing with a body, head, neck, and tail. You can customize the appearance of the horse by adjusting the dimensions of the shape, adding more details such as eyes, mane, and hooves, or using different shapes and styles.\n\nYou can also use the `pic` command to include a predefined horse shape in your drawing:\n\\documentclass{article}\n\\usepackage{tikz}\n\\usetikzlibrary{shapes.animals}\n\\'},
 {'role': 'user', 'content': 'END_THIS_NOW'}]

In [8]:
# allign spacy token boundaries with  embeddings with sentence piece embeddings and expose as the spacy.Token._.llama_vec Token property
import spacy
nlp = spacy.load("en_core_web_sm")

# pass spacy nlp object, result[:-1] (to avoid encoding END_THIS_NOW) and layer number (i.e. the layer from which to extract hidden state
doc = lc.get_spacy_doc(nlp,result[:-1],layer=0)

Returning encoding for:

--------------------

[INST] Can you use TiKZ code to draw a horse? [/INST]
 Yes, you can use TiKZ code to draw a horse. Here is an example of a simple horse drawing using TiKZ:

\documentclass{article}
\usepackage{tikz}
\usetikzlibrary{shapes,arrows,positioning}
\begin{document}
\beginIGROUP\draw (0,0) rectangle (2,2);% body
\draw (1,1) circle (0.5ex);% head
\draw (1,1.5) -- (2,0.5);% neck
\draw (0.5,1) -- (1.5,0);% tail
\endtikzpicture
\end{document}
This will produce a simple horse drawing with a body, head, neck, and tail. You can customize the appearance of the horse by adjusting the dimensions of the shape, adding more details such as eyes, mane, and hooves, or using different shapes and styles.

You can also use the `pic` command to include a predefined horse shape in your drawing:
\documentclass{article}
\usepackage{tikz}
\usetikzlibrary{shapes.animals}
\ 

--------------------


In [9]:
# demonstration of how to access Token._.llama_vec embedding
word_index = 9
print(f"""Average of sentence-piece embeddings for: `{doc[word_index]}`
at doc index: `{word_index}`
is of type: `{type(doc[5]._.llama_vec)}`
of shape: `{doc[5]._.llama_vec.shape}`""")

Average of sentence-piece embeddings for: `draw`
at doc index: `9`
is of type: `<class 'torch.Tensor'>`
of shape: `torch.Size([4096])`


In [51]:
# extract hidden states from result
hidden = lc.extract_embeddings(result[:-1],kind="hidden")
hidden[0].squeeze()

tensor([[ 0.0018, -0.0038,  0.0010,  ..., -0.0090,  0.0027, -0.0038],
        [ 0.0277, -0.0060,  0.0035,  ...,  0.0005, -0.0082,  0.0064],
        [-0.0299, -0.0148, -0.0530,  ...,  0.0047, -0.0173, -0.0120],
        ...,
        [ 0.0034, -0.0097,  0.0188,  ..., -0.0127, -0.0099, -0.0254],
        [-0.0344, -0.0071, -0.0178,  ..., -0.0126, -0.0008, -0.0535],
        [ 0.0003,  0.0029, -0.0126,  ...,  0.0014, -0.0019, -0.0064]],
       dtype=torch.float16, grad_fn=<SqueezeBackward0>)

In [47]:
# extract attentions from result
attentions = lc.extract_embeddings(result,kind="attention")
attentions[0].squeeze()[0]

tensor([[1.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [7.1777e-01, 2.8223e-01, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [5.6543e-01, 1.6431e-01, 2.7026e-01,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [1.7176e-03, 5.9891e-04, 8.7500e-04,  ..., 1.9054e-03, 0.0000e+00,
         0.0000e+00],
        [6.9737e-05, 1.8239e-05, 2.3186e-05,  ..., 1.8082e-03, 1.9263e-01,
         0.0000e+00],
        [3.5048e-05, 5.2452e-06, 1.0133e-05,  ..., 5.9509e-04, 2.4048e-02,
         7.6855e-01]], dtype=torch.float16, grad_fn=<SelectBackward0>)

In [48]:
# extract both from result
both = lc.extract_embeddings(result,kind="both")
both[0][0]

tensor([[[ 1.8387e-03, -3.8147e-03,  9.6130e-04,  ..., -9.0332e-03,
           2.6550e-03, -3.7537e-03],
         [ 2.7710e-02, -6.0425e-03,  3.4943e-03,  ...,  5.1117e-04,
          -8.1787e-03,  6.4087e-03],
         [-2.9907e-02, -1.4771e-02, -5.2979e-02,  ...,  4.7302e-03,
          -1.7334e-02, -1.2024e-02],
         ...,
         [-2.9907e-02, -1.4771e-02, -5.2979e-02,  ...,  4.7302e-03,
          -1.7334e-02, -1.2024e-02],
         [-2.3315e-02, -1.8677e-02,  5.8594e-03,  ...,  1.7578e-02,
          -2.0266e-05,  1.2390e-02],
         [-1.4954e-03, -3.5095e-04, -1.7624e-03,  ...,  8.8882e-04,
           1.6937e-03, -6.9427e-04]]], dtype=torch.float16,
       grad_fn=<ToCopyBackward0>)

In [10]:
#  print available GPT4 models
#authenticate openai with your key
openai.api_key = os.environ["OPENAI"]
[mod["id"] for mod in openai.Model.list()["data"] if mod["id"].startswith("gpt-4")]

['gpt-4-0314', 'gpt-4-0613', 'gpt-4']

In [14]:
# analgous chat class to LlamaChat but for GPT 4
# gpt-4 models
model_name = 'gpt-4-0314' #from march 14
# model_name = 'gpt-4-0613' #from june 13
# model_name = 'gpt-4' #most recent

# load chat class
gptc = plc.GptChat(model_name)


In [15]:
# chat with <<model_name>>, again input `END_THIS_NOW` to terminate the dialog and return the (List[Dict]) result
result = gptc(test_dialog)

User: Can you use TiKZ code to draw a horse?
Assistant: Yes, you can use TiKZ code to draw a horse, but it might be quite complex and time-consuming to create a detailed horse illustration using TiKZ. Here's a simple example of a horse drawing using TiKZ:

```latex
\documentclass{standalone}
\usepackage{tikz}

\begin{document}

\begin{tikzpicture}
    % Body
    \draw (0,0) ellipse (2 and 1);
    
    % Head
    \draw (2,0.5) -- (3,1) -- (3.5,0) -- (3,-1) -- (2,-0.5);
    \draw (3,1) -- (3.5,1.5);
    
    % Legs
    \draw (0.5,-1) -- (0.5,-3);
    \draw (1.5,-1) -- (1.5,-3);
    \draw (-0.5,-1) -- (-0.5,-3);
    \draw (-1.5,-1) -- (-1.5,-3);
    
    % Tail
    \draw (-2,0) -- (-3,1);
\end{tikzpicture}

\end{document}
```

This code will produce a very basic and minimalistic horse drawing. For more complex illustrations, you might want to consider using a vector graphics editor like Adobe Illustrator or Inkscape to create the horse drawing and then import it into your LaTeX document a

User:  END_THIS_NOW
